<style>div.title-slide {    width: 100%;    display: flex;    flex-direction: row;            /* default value; can be omitted */    flex-wrap: nowrap;              /* default value; can be omitted */    justify-content: space-between;}</style><div class="title-slide">
<span style="float:left;">Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat &amp; Arnaud Legout</span>
<span><img src="media/both-logos-small-alpha.png" style="display:inline" /></span>
</div>


# `DataFrame` en pandas

## Complément - niveau intermédiaire

### Création d'une DataFrame

Une DataFrame est un tableau numpy à deux dimension avec un index pour les lignes et un index pour les colonnes. Il y a de nombreuses manières de construire une DataFrame.

In [4]:
# Regardons la construction d'une DataFrame
import numpy as np
import pandas as pd

# Créons une Serie pour définir des ages
age = pd.Series([30, 20, 50], index=['alice', 'bob', 'julie'])

# et une Serie pour définir des tailles
height = pd.Series([150, 170, 168], index=['alice', 'marc', 'julie'])

# On peut maintenant combiner ces deux Series en DataFrame,
# chaque Series définissant une colonne, une manière de le faire est 
# de définir un dictionnaire qui contient pour clef le nom de la colonne
# et pour valeur la Series correspondante
stat = pd.DataFrame({'age': age, 'height':height})
print(stat)

        age  height
alice  30.0   150.0
bob    20.0     NaN
julie  50.0   168.0
marc    NaN   170.0


On remarque que pandas fait automatiquement l'alignement des index, lorsqu'une valeur n'est pas présente, elle est automatiquement remplacée par `NaN`. Pandas va également broadcaster une valeur unique définissant un colonne sur toutes les lignes. Regardons cela

In [6]:
stat = pd.DataFrame({'age': age, 'height':height, 'city': 'Nice'})
print(stat)

        age  city  height
alice  30.0  Nice   150.0
bob    20.0  Nice     NaN
julie  50.0  Nice   168.0
marc    NaN  Nice   170.0


In [7]:
# On peut maitenant accéder aux indexes des lignes et des colonnes

# l'index des lignes
print(stat.index)

# l'index des colonnes
print(stat.columns)

Index(['alice', 'bob', 'julie', 'marc'], dtype='object')
Index(['age', 'city', 'height'], dtype='object')


Il y a de nombreuses manières d'accéder maintenant aux éléments de la DataFrame, certaines sont bonnes et d'autres à proscrire, commençons par prendre de bonnes habitudes. Comme il s'agit d'une structure à deux dimensions, il faut donner un indice de ligne et de colonne.

In [25]:
# Quel est l'age de alice
a = stat.loc['alice', 'age']
print(f"l'age de alice est : {a}")

# Quel est la moyenne de tous les ages
m = stat.loc[:, 'age'].mean()
print(f"L'age moyen est de {m:.1f} ans")

l'age de alice est : 30.0
L'age moyen est de 33.3 ans


In [26]:
stat.loc[:, 'age'].mean()

33.333333333333336

On peut déjà noter plusieurs choses intéressantes

 - On peut utiliser `.loc[]` et `.iloc` comme pour les Series. Pour les DataFrame c'est encore plus important parce qu'il y a plus de risques d'ambiguïtés (notamment entre les lignes et le colonnes, on y reviendra). 
 - la méthode `mean` calcule la moyenne, ça n'est pas surprenant, mais ignore les `NaN`. C'est en général ce que l'on veut. Si vous vous demandez comment savoir si la méthode que vous utilisez ignore ou pas les `NaN`, le mieux est de regarder l'aide de cette méthode. Il existe pour un certain nombre de méthodes deux versions : une qui ignore les `NaN` et une autre qui les prend en compte&nbsp;; on reviendra dessus.

Une autre manière de construire une DataFrame est de partir d'un array numpy et de spécifier les indexes pour les lignes et les colonnes avec les arguments `index` et `columns`

In [28]:
a = np.random.randint(1, 20, 9).reshape(3,3)
p = pd.DataFrame(a, index=['a', 'b', 'c'], columns=['x', 'y', 'z'])
print(p)

    x  y   z
a   1  9  10
b   4  9  11
c  11  5   9


### Manipulation d'une DataFrame

In [179]:
# contruisons maintenant une DataFrame jouet

# voici une liste de prénoms
names = ['alice', 'bob', 'marc', 'bill', 'sonia']

# créons trois Series qui formeront trois colonnes
age = pd.Series([12, 13, 16, 11, 16], index=names)
height = pd.Series([130, 140, 176, 120, 165], index=names)
sex = pd.Series(list('fmmmf'), index=names)

# créons maintenant la DataFrame
p = pd.DataFrame({'age':age, 'height':height, 'sex':sex})
print(p)

       age  height sex
alice   12     130   f
bob     13     140   m
marc    16     176   m
bill    11     120   m
sonia   16     165   f


age     alice     12
        bob       13
        marc      16
        bill      11
        sonia     16
height  alice    130
        bob      140
        marc     176
        bill     120
        sonia    165
sex     alice      f
        bob        m
        marc       m
        bill       m
        sonia      f
dtype: object

In [32]:
# et chargeons le jeux de données sur les pourboires de seaborn
import seaborn as sns
tips = sns.load_dataset('tips')

Pandas offre de nombreuses possibilités d'explorer les données. Attention, dans mes exemples je vais alterner entre le DataFrame `p` et le DataFrame `tips` suivant les besoins de l'explication. 

In [33]:
# afficher les premières lignes 
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [34]:
# et les dernière lignes
tips.tail()

,total_bill,tip,sex,smoker,day,time,size
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2
243,18.78,3.00,Female,No,Thur,Dinner,2


In [35]:
# afficher l'index des lignes
p.index

Index(['alice', 'bob', 'marc', 'bill', 'sonia'], dtype='object')

In [36]:
# et de colonnes
p.columns

Index(['age', 'height', 'sex'], dtype='object')

In [37]:
# transposer 
p.T

,alice,bob,marc,bill,sonia
age,12,13,16,11,16
height,130,140,176,120,165
sex,f,m,m,m,f


In [38]:
# et afficher uniquement les valeurs
p.values

array([[12, 130, 'f'],
       [13, 140, 'm'],
       [16, 176, 'm'],
       [11, 120, 'm'],
       [16, 165, 'f']], dtype=object)

Pour finir, il y a la méthodes `describe` qui permet d'obtenir des premières statistiques sur un DataFrame. `describe` permet de calculer des statistiques sur des type numériques, mais aussi sur des types chaînes de caractères. 

In [42]:
# par défaut describe ne prend en compte que les colonnes numériques
p.describe()

,age,height
count,5.000000,5.000000
mean,13.600000,146.200000
std,2.302173,23.605084
min,11.000000,120.000000
25%,12.000000,130.000000
50%,13.000000,140.000000
75%,16.000000,165.000000
max,16.000000,176.000000


In [43]:
# mais on peut le forcer en prendre en compte toutes les colonnes
p.describe(include='all')

,age,height,sex
count,5.000000,5.000000,5
unique,NaN,NaN,2
top,NaN,NaN,m
freq,NaN,NaN,3
mean,13.600000,146.200000,NaN
std,2.302173,23.605084,NaN
min,11.000000,120.000000,NaN
25%,12.000000,130.000000,NaN
50%,13.000000,140.000000,NaN
75%,16.000000,165.000000,NaN


### Requêtes sur une DataFrame

On peut maintenant commencer à faire des requêtes sur les DataFrames. Les DataFrame supportent la notion de masque que l'on a vue pour les ndarray numpy et pour les Series. 

In [67]:
# p.loc prend soit un label de ligne
print(p.loc['sonia'])

age        16
height    165
sex         f
Name: sonia, dtype: object


In [68]:
# ou alors un label de ligne ET de colonne
print(p.loc['sonia', 'age'])

16


On peut mettre à la place d'une label :

 - une liste de labels
 - un slice sur les labels
 - un masque (c'est-à-dire un tableau de booléens)
 - un callable qui retourne une des trois premières possibilités
 
Noter que l'on peut également utiliser la notation `.iloc[]` avec les mêmes règles, mais elle est moins utile. 

Je recommande de toujours utiliser la notation `.loc[lignes, colonnes]` pour éviter toute ambiguïté. Nous verrons que les notations `.loc[lignes]` ou pire seulement `[label]` sont sources d'erreurs.

Regardons maintenant d'autres exemples plus sophistiqués.

In [61]:
# gardons uniquement les femmes
p.loc[p.loc[:,'sex']=='f',:]

,age,height,sex
alice,12,130,f
sonia,16,165,f


In [62]:
# gardons uniquement les femmes de plus de 12 ans
p.loc[(p.loc[:,'sex']=='f') & (p.loc[:, 'age'] > 12), :]

,age,height,sex
sonia,16,165,f


In [60]:
# quelle est la note moyenne des femmes
note_f = tips.loc[tips.loc[:,'sex']=='Female', 'total_bill'].mean()
print(f"note moyenne des femmes : {note_f:.2f}")

# quelle est la note moyenne des hommes
note_h = tips.loc[tips.loc[:,'sex']=='Male', 'total_bill'].mean()
print(f"note moyenne des hommes : {note_h:.2f}")

# qui laisse le plus grand pourcentage de pourboire : 
# les hommes ou les femmes

pourboire_f  = tips.loc[tips.loc[:,'sex']=='Female', 'tip'].mean()
pourboire_h  = tips.loc[tips.loc[:,'sex']=='Male', 'tip'].mean()

print(f"Les femmes laissent {pourboire_f/note_f:.2%} de pourboire")
print(f"Les hommes laissent {pourboire_h/note_h:.2%} de pourboire")

note moyenne des femmes : 18.06
note moyenne des hommes : 20.74
Les femmes laissent 15.69% de pourboire
Les hommes laissent 14.89% de pourboire


### Erreurs fréquentes et ambiguïtés sur les requêtes

Nous avons vu une manière simple et non ambiguë de faire des requêtes sur les DataFrame, mais nous allons voir qu'il existe d'autres manières qui ont pour seul avantage d'être plus concise, mais sources de nombreuses erreurs. 

**Souvenez-vous, utilisez toujours la notation `.loc[lignes, colonnes]` sinon, soyez sûr de savoir ce qui est réellement calculé**.

In [71]:
# commençons par la notation la plus classique
p['sex'] # prend forcément un label de colonne

alice    f
bob      m
marc     m
bill     m
sonia    f
Name: sex, dtype: object

In [72]:
# mais par contre, si on passe un slice, c'est forcément des lignes,
# assez perturbant et source de confusion.
p['alice': 'marc']

,age,height,sex
alice,12,130,f
bob,13,140,m
marc,16,176,m


In [74]:
# on peut même directement accéder à une colonne par son nom
p.age

alice    12
bob      13
marc     16
bill     11
sonia    16
Name: age, dtype: int64

In [99]:
# mais il ne faut jamais le faire parce que si un attribut de même 
# nom existe sur une DataFrame, alors la priorité est donnée à l'attribut
# et non à la colonne

# ajoutons une colonne qui a pour nom une méthode qui existe sur 
# les DataFrame
p['mean'] = 1
print(p)

       age  height sex  mean
alice   12     130   f     1
bob     13     140   m     1
marc    16     176   m     1
bill    11     120   m     1
sonia   16     165   f     1


In [100]:
# je peux bien accéder à la colonne sex
p.sex

alice    f
bob      m
marc     m
bill     m
sonia    f
Name: sex, dtype: object

In [101]:
# mais pas à la colonne mean
p.mean

<bound method DataFrame.mean of        age  height sex  mean
alice   12     130   f     1
bob     13     140   m     1
marc    16     176   m     1
bill    11     120   m     1
sonia   16     165   f     1>

In [102]:
# de nouveau, la seule méthode non ambiguë est d'utiliser .loc
p.loc[:,'mean']

alice    1
bob      1
marc     1
bill     1
sonia    1
Name: mean, dtype: int64

In [103]:
# supprimons maintenant la colonne mean *en place* (par défaut, 
# drop retourne une nouvelle DataFrame)
p.drop(columns='mean', inplace=True)
print(p)

       age  height sex
alice   12     130   f
bob     13     140   m
marc    16     176   m
bill    11     120   m
sonia   16     165   f


Pour allez plus loin, vous pouvez lire la documentation officielle : 

http://pandas.pydata.org/pandas-docs/stable/indexing.html

### Ufuncs et pandas

Ça n'est pas une surprise, les Series et DataFrame pandas supportent les Ufuncs numpy. Mais il y a une subtilité. Il est parfaitement légitime et correcte d'appliquer une Ufunc numpy sur les éléments d'une DataFrame

In [106]:
d = pd.DataFrame(np.random.randint(1, 10, 9).reshape(3,3), columns=list('abc'))
print(d)

   a  b  c
0  4  9  4
1  3  8  9
2  6  9  8


In [107]:
np.log(p/np.max(p))

,a,b,c
0,-2.197225,-1.791759,-0.117783
1,0.000000,-1.791759,-1.504077
2,-0.251314,0.000000,0.000000


Nous remarquons que comme on s'y attend, la Ufunc a été appliquée à chaque élément de la DataFrame et que les labels des lignes et colonnes ont été préservés. 

Par contre, si l'on a besoin d'alignement de labels, c'est le cas avec toutes les opérations qui s'appliquent sur 2 objets comme une addition, alors les Ufuncs numpy ne vont pas faire ce à quoi on s'attend. Elle vont faire les opérations sur les tableaux numpy sans prendre en compte les labels. 

Pour avoir un alignement des labels, il faut utiliser les Ufuncs pandas. 

In [108]:
# prenons deux series
s1 = pd.Series([10, 20, 30], index=list('abc'))
s2 = pd.Series([12, 22, 32], index=list('acd'))

# la Ufunc numpy fait la somme des arrays sans prendre en compte 
# les labels, donc sans alignement
np.add(s1, s2)

a    22
b    42
c    62
dtype: int64

In [109]:
# la Ufunc pandas va faire un alignement des labels
# cet appel est équivalent à s1 + s2
s1.add(s2)

a    22.0
b     NaN
c    52.0
d     NaN
dtype: float64

In [110]:
# comme on l'a vu sur le complément précédent, les valeurs absentes sont
# remplacées par NaN, mais on peut changer ce comportement lors de 
# l'appel de .add
s1.add(s2, fill_value=0)

a    22.0
b    20.0
c    52.0
d    32.0
dtype: float64

In [111]:
# regardons un autre exemple sur des DataFrame
names = ['alice', 'bob', 'charle']
bananas = pd.Series([10, 3, 9], index=names)
oranges = pd.Series([3, 11, 6], index=names)
fruits_jan = pd.DataFrame({'bananas':bananas, 'orange': oranges})

bananas = pd.Series([6, 1], index=names[:-1])
apples = pd.Series([8, 5], index=names[1:])
fruits_feb = pd.DataFrame({'bananas':bananas, 'apples': apples})

print(fruits_jan)
print(fruits_feb)

        bananas  orange
alice        10       3
bob           3      11
charle        9       6
        apples  bananas
alice      NaN      6.0
bob        8.0      1.0
charle     5.0      NaN


In [112]:
# regardons maintenant la somme des fruits mangés
eaten_fruits = fruits_jan + fruits_feb
print(eaten_fruits)

        apples  bananas  orange
alice      NaN     16.0     NaN
bob        NaN      4.0     NaN
charle     NaN      NaN     NaN


In [113]:
# On a bien un alignement des labels, mais il y a beaucoup de valeurs
# manquantes. Corrigeons cela on remplaçant les valeurs manquantes par 0
eaten_fruits = fruits_jan.add(fruits_feb, fill_value=0)
print(eaten_fruits)

        apples  bananas  orange
alice      NaN     16.0     3.0
bob        8.0      4.0    11.0
charle     5.0      9.0     6.0


Notons que lorsqu'une valeur est absente dans toutes les DataFrame, `NaN` est conservé.

Un dernière subtilité a connaître lors de l'alignement des labels est lorsque vous faites une opération sur une DataFrame et une Series. Pandas va considére la Series comme une ligne et va la broadcaster sur les autres lignes. Par conséquent, l'index de la Series va être considéré comme des colonnes et aligné avec les colonnes de la DataFrame.

In [134]:
d = pd.DataFrame(np.random.randint(1, 10, size=(3,3)), columns=list('abc'), index=list('xyz'))
s_row = pd.Series([-10, -10, -10], index=list('abc'))
s_col = pd.Series([-10, -10, -10], index=list('xyz'))
print(d)
print(s_row)
print(s_col)

   a  b  c
x  1  2  2
y  1  9  5
z  8  5  6
a   -10
b   -10
c   -10
dtype: int64
x   -10
y   -10
z   -10
dtype: int64


In [135]:
# la Series est considérée comme une ligne et son indexe 
# s'aligne sur les colonnes de la DataFrame. La Series va être
# broadcastée sur les autres lignes de la DataFrame

d + s_row
p

,a,b,c
0,1,1,8
1,9,1,2
2,7,6,9


In [136]:
# si les labels ne correspondent pas, le résultat sera le suivant

d + s_col

,a,b,c,x,y,z
x,NaN,NaN,NaN,NaN,NaN,NaN
y,NaN,NaN,NaN,NaN,NaN,NaN
z,NaN,NaN,NaN,NaN,NaN,NaN


In [137]:
# on peut dans ce cas, changer le comportement par défaut et forçant 
# l'alignement de la Series suivant un autre axe avec l'argument axis

d.add(s_col, axis=0)

,a,b,c
x,-9,-8,-8
y,-9,-1,-5
z,-2,-5,-4


Ici, `axis=0` signifie que la Series est considérée comme une colonne est qu'elle va être broadcasté sur les autres colonnes (le long de l'axe de ligne).

### Opérations sur les chaînes de caractères

Nous allons maintenant parler de la vectorisation des 
opérations sur les chaînes de caractères. Il y a plusieurs choses importantes à savoir.

 - Les méthodes sur les chaînes de caractères ne sont disponibles que pour les Series et les Index, mais pas pour les DataFrame. 
 - Ces méthodes ignorent les `NaN` et remplacent les valeurs qui ne sont pas des chaînes de caractères par `NaN`
 - Ces méthodes retournent une copie de l'objet (Series ou Index), il n'y a pas de modification en place
 - La plupart des méthodes Python sur le type `str` existe sous forme vectorisée
 - On accède à ces méthodes avec la syntaxe
   - `Series.str.<vectorized method name>`
   - `Index.str.<vectorized method name>`
   
Regardons quelques exemples.

In [139]:
# Créons une Series avec des noms ayant une capitalisation inconsistante
# et une mauvaise gestion des espaces
names = ['alice ', '  bOB', 'Marc', 'bill', 3, ' JULIE ', np.NaN]
age = pd.Series(names)

In [148]:
# nettoyons maintenant ces données

# on met en minuscule
a = age.str.lower()

# on enlève les espaces
a = a.str.strip()
print(a)

# comme les méthodes vectorisées retournent un objet de même type, on 
# peut les chainer ainsi

[x for x in age.str.lower().str.strip()]

0    alice
1      bob
2     marc
3     bill
4      NaN
5    julie
6      NaN
dtype: object


['alice', 'bob', 'marc', 'bill', nan, 'julie', nan]

In [150]:
# on peut également utiliser l'indexation des str de manière vectorisée
print(a)
print(a.str[-1])

0    alice
1      bob
2     marc
3     bill
4      NaN
5    julie
6      NaN
dtype: object
0      e
1      b
2      c
3      l
4    NaN
5      e
6    NaN
dtype: object


Pour allez plus loin vous pouvez lire la documentation officielle :

http://pandas.pydata.org/pandas-docs/stable/text.html

### Gestion des valeurs manquantes

Nous avons vu que des opérations sur les DataFrame pouvaient générer des valeurs `NaN` lors de l'alignement. Il est également possible d'avoir de telles valeurs _manquantes_ dans votre jeux de données original. Pandas offre plusieurs possibilités pour gérer correctement ces valeurs manquantes. 

Avant de voir ces différentes possibilités, définissons cette notion de valeur manquante.

Une valeur manquante peut-être représentée en pandas soit par `np.NaN` ou par l'objet Python `None`. 

 - `np.NaN` est un objet de type `float`, par conséquent il ne peut apparaître que dans un array de `float` ou un array d'`object`. Notons que `np.NaN` apparaît en pandas comme simplement `NaN` et que dans la suite on utilise de manière indifférente les deux notations, par contre, dans du code, il faut obligatoirement utiliser `np.NaN`.
     - si on ajoute un `NaN` dans un array d'entier, ils seront convertis en `float64`
     - si on ajoute un `NaN` dans un array de booléens, ils seront convertis en `object`
 - `NaN`est contaminant, toute opération avec un `NaN` a pour résultat `NaN`
 - lorsque l'on utilise `None`, il est automatiquement convertit en `NaN` lorsque le type de l'array est numérique.
 
Illustrons ces propriétés.
     

In [152]:
# une Series d'entier
s = pd.Series([1, 2])
print(s)

0    1
1    2
dtype: int64


In [153]:
# on ajouter un NaN, la Series est alors converties en float64
s[0] = np.NaN
print(s)

0    NaN
1    2.0
dtype: float64


In [154]:
# une nouvelle Serie d'entier
s = pd.Series([1, 2])

# et on ajouter None
s[0] = None

# None est converti en NaN
print(s)

0    NaN
1    2.0
dtype: float64


Regardons maintenant, les méthodes pandas pour gérer les valeurs manquantes (donc `NaN` ou `None`).

 - `isna()` retourne un masque mettant à True les valeurs manquantes (il y a un alias `isnull()`)
 - `notna()` retourne un masque mettant à False les valeurs manquantes (il y a un alias `notnull()`)
 - `dropna()` retourne un nouvel objet sans les valeurs manquantes
 - `fillna()` retourne un nouvel objet avec les valeurs manquantes remplacées
 
On remarque que l'ajout d'alias pour les méthodes est de nouveau une source de confusion avec laquelle il faut vivre. 

On remarque également qu'alors que `isnull()` et `notnull()` sont des méthodes simples, `dropna()` et `fillna()` impliquent l'utilisation de stratégies. Regardons cela. 

In [156]:
# créons une DataFrame avec quelques valeurs manquantes
names = ['alice', 'bob', 'charle']
bananas = pd.Series([6, 1], index=names[:-1])
apples = pd.Series([8, 5], index=names[1:])
fruits_feb = pd.DataFrame({'bananas':bananas, 'apples': apples})
print(fruits_feb)

        apples  bananas
alice      NaN      6.0
bob        8.0      1.0
charle     5.0      NaN


In [158]:
fruits_feb.isna()

,apples,bananas
alice,True,False
bob,False,False
charle,False,True


In [162]:
fruits_feb.notna()

,apples,bananas
alice,False,True
bob,True,True
charle,True,False


Par défaut, `dropna()` va enlever toutes les lignes qui contiennent au moins une valeur manquante. Mais on peut changer ce comportement avec des arguments. Regardons quelques exemples.

In [164]:
p = pd.DataFrame([[1, 2, np.NaN], [3, np.NaN, np.NaN], [7, 5, np.NaN]])
print(p)

   0    1   2
0  1  2.0 NaN
1  3  NaN NaN
2  7  5.0 NaN


In [165]:
# comportement par défaut, j'enlève toutes les lignes avec au moins 
# une valeur manquante
p.dropna()

,0,1,2


In [166]:
# maintenant, je fais l'opération par colonne
p.dropna(axis=1)

,0
0,1
1,3
2,7


In [167]:
# je vais l'opération par colonne qui si toute la colonne est manquante
p.dropna(axis=1, how='all')

,0,1
0,1,2.0
1,3,NaN
2,7,5.0


In [168]:
# je fais l'opération par ligne si au moins 2 valeurs sont manquantes
p.dropna(thresh=2)

,0,1,2
0,1,2.0,NaN
2,7,5.0,NaN


Par défaut, `fillna()` remplace les valeurs manquantes avec un argument pas défaut. Mais on peut ici aussi changer ce comportement. Regardons cela.

In [169]:
print(p)

   0    1   2
0  1  2.0 NaN
1  3  NaN NaN
2  7  5.0 NaN


In [170]:
# je remplace les valeurs manquantes par -1
p.fillna(-1)

,0,1,2
0,1,2.0,-1.0
1,3,-1.0,-1.0
2,7,5.0,-1.0


In [172]:
# je remplace les valeurs manquantes avec la valeur suivante sur la colonne
# bfill est pour back fill, c'est-à-dire remplace en arrière à partir des
# valeurs existante
p.fillna(method='bfill')

,0,1,2
0,1,2.0,NaN
1,3,5.0,NaN
2,7,5.0,NaN


In [174]:
# je remplace les valeurs manquantes avec la valeur précédente sur la ligne
# ffill est pour forward fill, remplace en avant à partir des valeurs
# existantes
p.fillna(method='ffill', axis=1)

,0,1,2
0,1.0,2.0,2.0
1,3.0,3.0,3.0
2,7.0,5.0,5.0


Regardez l'aide de ces méthodes pour aller plus loin.

In [177]:
p.dropna?

In [178]:
p.fillna?

### Les MultiIndex

Pandas avait historiquement d'autres structures de données en plus des Series et des DataFrame permettant d'exprimer des dimensionalités supérieur à 2 comme par exemple les Panel. Mais pour des raisons de maintenance du code et d'optimisations, les développeurs ont décidé de ne garder que les Series et les DataFrame. Alors, comment exprimer des données avec plus de deux dimensions ?

On utilise pour cela des MultiIndex. Un MultiIndex, est un index qui peut être utilisé partout où l'on utilise un Index (dans une Series, ou comme ligne ou colonne d'une DataFrame) et qui a pour caractéristiques d'avoir plusieurs niveaux. Regardons tout de suite un exemple.

In [180]:
# contruisons une DataFrame jouet

# voici une liste de prénoms
names = ['alice', 'bob', 'sonia']

# créons trois Series qui formeront trois colonnes
age = pd.Series([12, 13, 16], index=names)
height = pd.Series([130, 140, 165], index=names)
sex = pd.Series(list('fmf'), index=names)

# créons maintenant la DataFrame
p = pd.DataFrame({'age':age, 'height':height, 'sex':sex})
print(p)

       age  height sex
alice   12     130   f
bob     13     140   m
sonia   16     165   f


In [182]:
s = p.unstack()
print(s)

age     alice     12
        bob       13
        sonia     16
height  alice    130
        bob      140
        sonia    165
sex     alice      f
        bob        m
        sonia      f
dtype: object


### Acquisition de données